In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%reload_ext autoreload

In [109]:
import logging
import csv
import sys
from pathlib import Path
import subprocess


In [4]:
logger = logging.getLogger(__name__)
logger.root.setLevel('DEBUG')

In [59]:
def csv_to_list(file):
    '''read csv file `file` into a list
    
    Guess the CSV dialect (e.g. tsv, csv, etc.)
    
    Returns `list`'''
    csvFile = Path(file).expanduser().absolute()
    file_csv = []
    
    with open(csvFile, 'r') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
        file.seek(0)
        reader = csv.reader(file, dialect)
        for row in reader:
            file_csv.append(row)

    return file_csv

In [61]:
def map_headers(csv_list, expected_headers=[]):
    '''map row 0 of a csv as formatted as a list to a dictionary of expected header values'''
    missing_headers = []
    header_map = {}
    
    csvHeader = csv_list[0]
    
    logger.debug('checking for missing headers')
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
            
    if len(missing_headers) > 0:
        logging.warning(f'missing expected headers: {missing_headers}')
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            header_map[value] = index
        
    logging.debug('completed mapping')
    return(header_map, missing_headers)

In [16]:
gd = Path('/Volumes/GoogleDrive/Shared drives/IT Blabla I/')

In [89]:
def do_exit(e='unknown error in unknown module: BSoD!', exit_status=0, testing=False):
    logging.info(f'exited before completion with exit code {exit_status}')
    print('program exited due to errors')
    print(e)
    if not testing:
        sys.exit(exit_status)

In [119]:
p = subprocess.Popen('xattr -p user.drive.itemprotostr "/Volumes/GoogleDrive/Shared drives/IT Blabla I/foo.txt"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

In [120]:
j = ''
for each in p.stdout.readlines():
    for

b'id: "1YfYhQWnmlOnZIn41Wnp2HgDf760JLTwM"\n'
b'parent: "0ACLfU8KeD_BHUk9PVA"\n'
b'title: "foo.txt"\n'
b'mime_type: "text/plain"\n'
b'starred: false\n'
b'trashed: false\n'
b'create_date_millis: 1539342132044\n'
b'modified_date_millis: 1539342144059\n'
b'last_viewed_by_me_date_millis: 1539342144059\n'
b'file_size: 30\n'
b'explicitly_trashed: false\n'
b'subscribed: false\n'
b'has_thumbnail: true\n'
b'file_extension: "txt"\n'
b'primary_sync_parent: ""\n'
b'md5_checksum: "ca2ee7374a46a4a0efa1bd2c1ccd3731"\n'
b'property {\n'
b'  key: "local-title"\n'
b'  string_value: "foo.txt"\n'
b'}\n'
b'property {\n'
b'  key: "version-counter"\n'
b'  int64_value: 4\n'
b'}\n'
b'property {\n'
b'  key: "content-entry"\n'
b'  binary_value: "\\010\\261\\243\\n\\02230B9WTleJ1MzaYVExMSmxtdkduQzZBQ3ZvTUM0Z1hTL1FFS29jPQ\\032!1YfYhQWnmlOnZIn41Wnp2HgDf760JLTwM \\036"\n'
b'}\n'
b'etag: "\\"1539342144059\\""\n'
b'editable: true\n'
b'can_comment: true\n'
b'server_create_date_millis: 1539342139768\n'
b'team_drive_id: "0

In [103]:
class gd_path():
    def __init__(self, path=None, drive_id=None, url=None):
        self.path = path
        self.drive_id = drive_id
        self.url = url
    
    @property
    def path(self):
        return self._path
    
    @path.setter
    def path(self, path):
        self._path = Path(path).expanduser().absolute()
        
        
    

In [104]:
g = gd_path('/Volumes/GoogleDrive/Shared drives/IT Blabla I/foo')

In [107]:
g.path

PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I/foo')

In [94]:
def main():
    
    
    csv_file = Path('./student.export.csv.text')
    
    drive_path = Path('/Volumes/GoogleDrive/Shared drives/IT Blabla I/')
    
    expected_headers = ['LastFirst', 'ClassOf', 'Student_Number']
    expected_headers ={'LastFirst': str, 'ClassOf': int, 'Student_Number': int}
    
    ############
    
    logging.debug('starting up')
    try:
        csv_list = csv_to_list(csv_file)
    except (FileNotFoundError, OSError, IOError) as e:
        logging.error(f'could not read csv file: {csv_file}')
        logging.error(f'{e}')
        do_exit(e, 1)
    
    header_map, missing_headers = map_headers(csv_list, expected_headers.keys())
    
    if len(missing_headers) > 0:
        do_exit(f'{csv_file.name} is missing one or more headers:\n\t{missing_headers}', 1)
    
    
    return
    


In [95]:
f = main()

DEBUG:root:starting up
DEBUG:__main__:checking for missing headers
DEBUG:root:completed mapping


In [75]:
print( f)

[['LastFirst', 'U_StudentsUserFields.emailstudent', 'Last_Name', 'First_Name', 'GuardianEmail', 'Grade_Level', 'Student_Number', 'StudentCoreFields.fatherdayphone', 'U_StudentsUserFields.father_mobile', 'U_StudentsUserFields.mother_mobile', 'Home_Phone', 'ClassOf'], ['Wanja, Michelle', 'mwanja@ash.nl', 'Wanja', 'Michelle', 'info@kenyaembassy.nl', '9', '505586', '', '', '+31617226808', '070.406.5427', '2023'], ['Fordney, Joseph', 'jfordney@ash.nl', 'Fordney', 'Joseph', 'llfordney@gmail.com', '10', '505567', '', '+31683014387', '+31615182071', '000.000.0000', '2022'], ['Jaber, Fahad', 'fjaber@ash.nl', 'Jaber', 'Fahad', 'a.al-mahmoud@hotmail.com', '10', '505564', '', '+31633337711', '+31633338808', '000.000.0000', '2022'], ['Lillie, Cameron', 'clillie@ash.nl', 'Lillie', 'Cameron', 'julie.j.alexander@shell.com', '11', '505590', '', '+27793990604', '+31627269945', '000.000.0000', '2021']]
